<a href="https://colab.research.google.com/github/matteogianferrari/homelens-ca/blob/main/model/nottebooks/homelens_ca_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **HomeLens CA DL Model Research**

# **Setup**


---
**OS Environment Variables**

In [11]:
from google.colab import userdata
import os

# Sets environ variables for GitHub
os.environ['GITHUB_TOKEN'] = userdata.get('GITHUB_TOKEN')
os.environ['USER'] = userdata.get('USER')

# Sets environ variables for MLFlow
os.environ['MLFLOW_TRACKING_USERNAME'] = userdata.get('USER')
os.environ['MLFLOW_TRACKING_PASSWORD'] = userdata.get('MLFLOW_TRACKING_PASSWORD')

---
**Libraries Install**

In [12]:
!pip install dagshub mlflow pynvml --quiet

---
**Libraries imports**

In [13]:
import tensorflow as tf

import mlflow
from mlflow.tracking import MlflowClient

import dagshub

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

---
**Dataset import**

In [14]:
# Clones the repo and changes dir
!git clone https://${GITHUB_TOKEN}@github.com/${USER}/homelens-ca.git
%cd homelens-ca/

Cloning into 'homelens-ca'...
remote: Enumerating objects: 190, done.
remote: Counting objects: 100% (190/190), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 190 (delta 97), reused 144 (delta 61), pack-reused 0 (from 0)
Receiving objects: 100% (190/190), 7.35 MiB | 23.75 MiB/s, done.
Resolving deltas: 100% (97/97), done.
/content/homelens-ca/model/data/dataset/homelens-ca


In [15]:
# Changes dir
%cd model/data/dataset

# Imports the dataset and the labels
X = pd.read_csv('X.csv').to_numpy()
y = pd.read_csv('y.csv').to_numpy()

# Applies transformation to labels
y = np.log1p(y)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

/content/homelens-ca/model/data/dataset/homelens-ca/model/data/dataset


---
**DagsHub and MLFlow**

In [16]:
# Init DagsHub
dagshub.init(repo_owner='matteogianferrari', repo_name='homelens-ca', mlflow=True)
TRACKING_URI = "https://dagshub.com/matteogianferrari/homelens-ca.mlflow"

# Sets MLFlow tracking URI
mlflow.set_tracking_uri(TRACKING_URI)

# Sets MLFLow experiment name
experiment_name = "HomeLens CA-DL Research"

Initialized MLflow to track repo "matteogianferrari/homelens-ca"

Repository matteogianferrari/homelens-ca initialized!

# **Research and Training DL Models**

---
**Model Hyperparameters**

Defines the Hyperparameters for the model.

In [17]:
epochs = 500
initial_learning_rate = 1e-3
minimum_learning_rate = 1e-8
patience_es=50
patience_lr=15
decaying_factor=0.5
batch_size=128
validation_slip=0.1
loss_func='mse'

---
**Training and Testing**

In [18]:
import pynvml
from tensorflow.keras.regularizers import l2

# Sets the MLFlow experiment
mlflow.set_experiment(experiment_name)

# Autolog
mlflow.keras.autolog()



def residual_block(x, dropout_rate=0.3, l2_reg=1e-4):
    """
    Implements a residual block with the following series:
      Dense(512) -> BN -> Dropout ->
      Dense(256) -> BN -> Dropout ->
      Dense(128) -> BN -> Dropout ->
      (Add shortcut) -> ReLU activation
    """
    # Save the input for the residual (skip) connection.
    shortcut = x

    # First dense layer: 512 units.
    x = tf.keras.layers.Dense(512, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)

    # Second dense layer: 256 units.
    x = tf.keras.layers.Dense(256, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)

    # Third dense layer: 128 units.
    x = tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)

    # Before adding, ensure the shortcut has the same dimension (128) as the output.
    if shortcut.shape[-1] != 128:
        shortcut = tf.keras.layers.Dense(128, kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg))(shortcut)

    # Add the shortcut (residual connection) and apply a ReLU activation.
    x = tf.keras.layers.Add()([x, shortcut])
    x = tf.keras.layers.Activation('relu')(x)
    return x

# Define the input layer for 15 features.
inputs = tf.keras.layers.Input(shape=(15,))

# Apply the residual block.
x = residual_block(inputs, dropout_rate=0.2)
x = residual_block(x, dropout_rate=0.2)
x = residual_block(x, dropout_rate=0.2)
x = residual_block(x, dropout_rate=0.2)

# Optionally, add additional layers after the residual block.
x = tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(x)

# Final output layer for regression (one continuous output).
outputs = tf.keras.layers.Dense(1)(x)

# Create and compile the model.
model = tf.keras.Model(inputs=inputs, outputs=outputs)



# Set up a decaying learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate)

# You can now compile your model using this optimizer
model.compile(
    optimizer=optimizer,
    loss=loss_func,
    metrics=['r2_score', 'mae']
)

# Creates an Early-Stopping callback
es_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    mode='min',
    patience=patience_es,
    restore_best_weights=True
)

lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=decaying_factor,
    patience=patience_lr,
    min_lr=minimum_learning_rate
)


# Train the model and log with MLFlow
with mlflow.start_run(log_system_metrics=True):
    mlflow.log_param("patience", patience_es)
    mlflow.log_param("loss_func", loss_func)

    history = model.fit(
        X_train,
        y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_split=validation_slip,
        callbacks=[es_callback, lr_callback]
    )

    # Test the model and log with MLFlow
    test_loss, test_r2_score, test_mae = model.evaluate(X_test, y_test)
    mlflow.log_metric("test_loss", test_loss)
    mlflow.log_metric("test_r2_score", test_r2_score)
    mlflow.log_metric("test_mae", test_mae)

2025/02/02 14:54:52 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Epoch 1/500
116/116 ━━━━━━━━━━━━━━━━━━━━ 30s 109ms/step - loss: 40.8155 - mae: 4.5339 - r2_score: -141.6601 - val_loss: 17.2039 - val_mae: 4.0215 - val_r2_score: -60.3650 - learning_rate: 0.0010
Epoch 2/500
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 2.1858 - mae: 0.9435 - r2_score: -4.0962 - val_loss: 4.8671 - val_mae: 1.9552 - val_r2_score: -14.3595 - learning_rate: 0.0010
Epoch 3/500
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 1.8400 - mae: 0.8264 - r2_score: -2.9658 - val_loss: 2.3688 - val_mae: 1.1582 - val_r2_score: -5.0461 - learning_rate: 0.0010
Epoch 4/500
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 1.6575 - mae: 0.7582 - r2_score: -2.3463 - val_loss: 2.3879 - val_mae: 1.1714 - val_r2_score: -5.1210 - learning_rate: 0.0010
Epoch 5/500
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 1.5029 - mae: 0.6899 - r2_score: -1.7579 - val_loss: 1.8440 - val_mae: 0.9531 - val_r2_score: -3.0960 - learning_rate: 0.0010
Epoch 6/500
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - l

2025/02/02 15:05:10 ERROR mlflow.utils.async_logging.async_logging_queue: Run Id 766d936c615a47dc864b85f8f26a3634: Failed to log run data: Exception: API request to https://dagshub.com/matteogianferrari/homelens-ca.mlflow/api/2.0/mlflow/runs/log-batch failed with exception HTTPSConnectionPool(host='dagshub.com', port=443): Max retries exceeded with url: /matteogianferrari/homelens-ca.mlflow/api/2.0/mlflow/runs/log-batch (Caused by ResponseError('too many 429 error responses'))
2025/02/02 15:11:23 WARNING mlflow.system_metrics.system_metrics_monitor: Failed to get mlflow run: API request to https://dagshub.com/matteogianferrari/homelens-ca.mlflow/api/2.0/mlflow/runs/get failed with exception HTTPSConnectionPool(host='dagshub.com', port=443): Max retries exceeded with url: /matteogianferrari/homelens-ca.mlflow/api/2.0/mlflow/runs/get?run_uuid=766d936c615a47dc864b85f8f26a3634&run_id=766d936c615a47dc864b85f8f26a3634 (Caused by ResponseError('too many 429 error responses')).


🏃 View run suave-sow-341 at: https://dagshub.com/matteogianferrari/homelens-ca.mlflow/#/experiments/0/runs/766d936c615a47dc864b85f8f26a3634
🧪 View experiment at: https://dagshub.com/matteogianferrari/homelens-ca.mlflow/#/experiments/0


2025/02/02 15:11:51 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/02/02 15:11:51 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
